# Algorytmika i matematyka uczenia maszynowego 
## Laboratorium 12

### Zadanie 1

Zaimplementuj systemu rekomendacji filmów w oparciu o indeks Jaccarda. System ma za zadanie zwrócić listę filmów sugerowany dla podanego użytownika.

Dane zostały pobrane z serwisu Kaggle z https://www.kaggle.com/datasets/gargmanas/movierecommenderdataset

Zbiór zawiera dwa pliki:
- `movies.csv` lista filmów wraz z ich identyfikatorami
- `ratings.csv` lista ocen filmów przez użytkowników

Wykonaj:
* Wczytaj oba pliki.
* Zamień wszystkie oceny użytkowników na wartości binarne, np. 1 jeżeli oceniony pozytywnie (zastosuj próg okreśjący np. ocena >3 oznacza pozytywną ocenę).
* Stwórz macierz ocen użytkowników w której wiersze będą reprezentowac użytkow, a w kolumny filmy. Wartość w macierzy jest flagą mówiącą czy użytkownik ocenił film pozytywnie czy tez nie. 
* Wypełnij brakujące wartości zerami.
* Utwórz macierz podobieństwa Jaccarda pomiędzy użytkownikami (każdy z każdym).
    - Możesz wykorzystać funkcję [jaccard](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.jaccard.html) z biblioteki scipy.
* Zaimplementuj funkcję która dla podanego użytkownika zwróci listę sugerowanych filmów.
    - Wybierz $n$ najbardziej podobnych użytkownikow.
    - Sprawdz jakie filmy najczesciej wystepuja w zbiorze i zwroc je.
    - Pamietaj: funkcja powinna zwrócić listę filmów które nie były ocenione przez użytkownika wczesniej.



In [28]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances

movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

ratings['binary_rating'] = ratings['rating'].apply(lambda x: 1 if x > 3 else 0)

user_movie_matrix = ratings.pivot_table(index='userId', columns='movieId', values='binary_rating', fill_value=0)

user_movie_matrix_np = user_movie_matrix.to_numpy()

user_similarity = 1 - pairwise_distances(user_movie_matrix_np, metric='jaccard')



def recommend_movies(user_id, user_movie_matrix, user_similarity, n_neighbors=5):
    similar_users = user_similarity[user_id - 1].argsort()[-(n_neighbors + 1):-1]
    similar_users_ratings = user_movie_matrix.iloc[similar_users]

    user_ratings = user_movie_matrix.iloc[user_id - 1]

    movie_recommendations = similar_users_ratings.sum(axis=0)
    movie_recommendations = movie_recommendations[user_ratings == 0]
    recommended_movies = movie_recommendations.sort_values(ascending=False).index.tolist()
    return recommended_movies

recommended_movies = recommend_movies(user_id=1, user_movie_matrix=user_movie_matrix, user_similarity=user_similarity, n_neighbors=5)
print("Rekomendowane filmy dla użytkownika 1:", recommended_movies[:10])


c:\Users\Dominik\anaconda3\envs\amumu\Lib\site-packages\sklearn\metrics\pairwise.py:2317: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Rekomendowane filmy dla użytkownika 1: [1036, 1200, 1259, 589, 1387, 2194, 2791, 1663, 377, 2683]




### Zadanie 2


Algorytm MinHash na przykładzie wykrywania plagiatów

Wykonaj kolejno następujące kroki:

1. Pobierz 8 akapitów tekstu (nie za krótkich), każdy o różnej tematyce (mogą być np. z różnych haseł Wikipedii), trzymaj się jednego języka (np. PL lub ENG). Wklej je do jednego pliku tekstowego, z linią wolną jako separatorem.

2. Skopiuj wybrane 2-3 akapity i ręcznie nieco zmodyfikuj.

> Przykład (z hasła https://pl.wikipedia.org/wiki/Fryderyk_Chopin):

```Jest uważany za jednego z najwybitniejszych kompozytorów romantycznych, a także za jednego z najważniejszych polskich kompozytorów w historii. Był jednym z najsłynniejszych pianistów swoich czasów, często nazywany poetą fortepianu. Elementem charakterystycznym dla utworów Chopina jest pogłębiona ekspresja oraz czerpanie z wzorców stylistycznych polskiej muzyki ludowej.```

↓↓↓ Zmieniono na ↓↓↓

```Jest uważany za jednego z najwybitniejszych kompozytorów romantycznych, a także za jednego z najważniejszych kompozytorów polskich w historii.  Był jednym  z najsłynniejszych pianistów swoich czasów, często nazywany poetą fortepianu! Elementem charakterystycznym dla utworów Fryderyka Chopina jest pogłębiona ekspresja oraz czerpanie z wzorców polskiej muzyki ludowej.```

Otrzymasz zatem w pliku tekstowym 10 lub 11 akapitów tekstu (kolejność dowolna, te "splagiatowane" nie muszą być na końcu).

3. Z poziomu skryptu: wczytaj wszystkie akapity z pliku. Zbuduj 100 "losowych" funkcji haszujących.

> Sugestia: funkcją "bazową" jest po prostu `hash(...)`. Zakładamy 64-bitową wersję Pythona 3.x, wtedy `hash(...)` jest 64-bitowy.
Na liście seeds umieszczamy 100 losowych liczb 64-bitowych. Aby obliczyć $i$-ty hash dla ciągu 
należy wykonać `hash(s) ^ seeds[i]` (użycie operatora XOR).

4. Przyjmij niewielką wartość $Q$ (np. 15), nastepnie dla każdego akapitu (zakladajac ze jego długość wynosi $n$) i dla każdej ze 100 funkcji haszujących policz hasza w przesuwnym oknie tekstu o długości $Q$ znaków (łącznie mamy $n - Q + 1$ wartości hasza); zapamiętaj MINIMUM z tych $n - Q + 1$ wartości.
Na wyjściu mamy zatem (dla 11 akapitów) 11 * 100 wartości haszy.

5. Rozważ pary akapitów "każdy z każdym". Jeśli dla danej pary co najmniej (np.) 30 haszy jest wspólnych, to uważamy akapity za podobne (być może plagiat) i wyświetlamy na ekranie.

6. Wyświetl czas obliczeń (powinien wynosić mniej niż 0.5s).

7. Poeksperymentuj z liczbą użytych funkcji haszujących, wartością, stopniem modyfikacji oryginalnych akapitów tekstu, progiem detekcji akapitów podobnych.

In [12]:
import numpy as np
import random

def generate_seeds(num_seeds=100):
    random.seed(254397)
    seeds = [random.getrandbits(64) for _ in range(num_seeds)]
    return seeds


def calculate_min_hashes(paragraphs, seeds, Q=15):
    min_hashes = []

    for paragraph in paragraphs:
        n = len(paragraph)
        para_hashes = []

        for seed in seeds:
            q_hashes = [
                hash(paragraph[i:i+Q]) ^ seed for i in range(n - Q + 1)
            ]
            min_hash = min(q_hashes)
            para_hashes.append(min_hash)

        min_hashes.append(para_hashes)

    return min_hashes

def find_similar_paragraphs(min_hashes, threshold=30):
    num_paragraphs = len(min_hashes)
    similar_pairs = []

    for i in range(num_paragraphs):
        for j in range(i + 1, num_paragraphs):
            common_hashes = sum(1 for a, b in zip(min_hashes[i], min_hashes[j]) if a == b)
            if common_hashes >= threshold:
                similar_pairs.append((i, j))
    return similar_pairs


with open('akapity.txt', 'r', encoding='utf-8') as f:
    text = f.read()
paragraphs = text.split('\n')

seeds = generate_seeds()
Q = 15

min_hashes = calculate_min_hashes(paragraphs, seeds, Q)
similar_pairs = find_similar_paragraphs(min_hashes)

print("Podobne akapity (indeksy):")
for i, j in similar_pairs:
    print(f"Akapity {i} i {j} są podobne")

Podobne akapity (indeksy):
Akapity 4 i 6 są podobne
Akapity 5 i 7 są podobne
